# Asymptotic approximations

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sandialabs/Polymers/main?labpath=docs%2Fsource%2F%2Fphysics%2Fsingle_chain%2Ffjc%2Fthermodynamics%2Fmodified_canonical%2Fexample_asymptotic.ipynb)

This example demonstrates the validity of the asymptotic approaches of approximating the thermodynamics of the freely-jointed chain (FJC) model in the modified canonical ensemble. To start, import and create an instance of the FJC model in the modified canonical ensemble:

In [ ]:
from polymers import physics
fjc = physics.single_chain.fjc.thermodynamics.modified_canonical.FJC(8, 1, 1)

## Strong potential

For sufficiently strong potentials, the modified canonical ensemble can be accurately approximated using the reference system (the isometric ensemble) and an asymptotic correction. For example, the nondimensional force $\eta$ as a function of the nondimensional potential distance $\gamma$ is approximated as

$$
  \eta(\gamma) \sim \eta_0(\gamma) - \frac{1}{\kappa}\left[N_b\eta_0(\gamma)\eta_0'(\gamma) - \frac{\eta_0''(\gamma)}{2}\right]\quad\text{for }\kappa\gg 1,
$$

where $\kappa$ is the nondimensional potential stiffness, $N_b$ is the number of links, and the nondimensional force $\eta_0$ for the freely-jointed chain model in the isometric ensemble as a function of the nondimensional end-to-end length per link $\gamma$ is given by

$$
  \eta_0(\gamma) = \frac{1}{N_b\gamma} + \left(\frac{1}{2} - \frac{1}{N_b}\right)\frac{\sum_{s=0}^{s_\mathrm{max}}(-1)^s\binom{N_b}{s}\left(\frac{1-\gamma}{2} - \frac{s}{N_b}\right)^{N_b - 3}}{\sum_{s=0}^{s_\mathrm{max}}(-1)^s\binom{N_b}{s}\left(\frac{1-\gamma}{2} - \frac{s}{N_b}\right)^{N_b - 2}}\quad\text{where }s_\mathrm{max}\leq N_b m\leq s_\mathrm{max}+1.
$$

In contrast, the exact relation for $\eta(\gamma)$ is obtained by numerically evaluating

$$
  \eta(\gamma) = -\frac{1}{N_b}\frac{\partial}{\partial\gamma}\ln\left[\iiint Q_0(\boldsymbol{\gamma}') e^{-\frac{\kappa}{2}\left(\boldsymbol{\gamma} - \boldsymbol{\gamma}'\right)^2} \,d^3\boldsymbol{\gamma}'\right],
$$

where $Q_0$ is the partition function in the isometric ensemble. This exact relation is plotted below along with the asymptotic relation while varying $\kappa$, the nondimensional potential stiffness. As $\kappa$ increases, the asymptotic approach appears to do increasingly well, but always eventually fails as $\gamma$ becomes large. This is because the asymptotic approach in this case is completely invalid for larger values of $\gamma$ (approaching 1), where the massive associated entropic force would cause the end of the chain to leave the bottom of the potential well, even though it may be quite strong.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
kappa_list = [1e2, 1e3, 1e4]
gamma = np.linspace(1e-3, 99e-2, 100)
for kappa in kappa_list:
    eta = fjc.nondimensional_force(gamma, kappa)
    line = plt.plot(gamma, eta, label=r'$\kappa=$' + str(kappa))
    eta_asymptotic = fjc.asymptotic.strong_potential.nondimensional_force(gamma, kappa)
    plt.plot(gamma, eta_asymptotic, ':', color=line[0].get_color())
plt.legend()
plt.xlim([0, 1])
plt.ylim([0, 12])
plt.xlabel(r'$\gamma$')
plt.ylabel(r'$\eta$')
plt.show()

To quantitatively evaluate the asymptotic approximation $\eta_a$, the relative $L_2$ error is computed as a function of the nondimensional potential stiffness $\kappa$,

$$
  e(\kappa) = \sqrt{\frac{\int\left[\eta(\gamma) - \eta_a(\gamma)\right]^2 d\gamma}{\int\eta^2(\gamma) \,d\gamma}}.
$$

When integrating over the whole domain $\gamma\in(0,1)$, the relative error $e$ is quite high and decreases with a log-log slope of around $-1$. This is because the asymptotic approximation is only valid for smaller values of $\gamma$, and only more intermediate values as $\kappa$ increases (as shown above) and/or $N_b$ decreases. When integration over the first third of the domain, $\gamma\in(0,1/3)$, the relative error $e$ is quite low and decreases with a log-log slope of around $-2$. The low error and log-log slope indicate that the asymptotic approximation is correct to within $\mathrm{ord}(\kappa^{-2})$ for large values of $\kappa$, as expected from the asymptotic theory.

In [ ]:
plt.subplot(1, 2, 1)
gamma = np.linspace(1e-3, 99e-2, 100)
relative_error = np.zeros(len(gamma))
kappa = np.logspace(1, 3, len(gamma))
for i, kappa_i in enumerate(kappa):
    eta = fjc.nondimensional_force(gamma, kappa_i)
    eta_asymptotic = fjc.asymptotic.strong_potential.nondimensional_force(gamma, kappa_i)
    error = np.sqrt(np.trapz((eta - eta_asymptotic)**2, x=gamma))
    relative_error[i] = error/np.sqrt(np.trapz(eta**2, x=gamma))
plt.loglog(kappa, relative_error)
plt.xlabel(r'$\kappa$')
plt.ylabel(r'$e$ for $\gamma\in(0,1)$')

plt.subplot(1, 2, 2)
gamma = np.linspace(1e-3, 1/3, 100)
for i, kappa_i in enumerate(kappa):
    eta = fjc.nondimensional_force(gamma, kappa_i)
    eta_asymptotic = fjc.asymptotic.strong_potential.nondimensional_force(gamma, kappa_i)
    error = np.sqrt(np.trapz((eta - eta_asymptotic)**2, x=gamma))
    relative_error[i] = error/np.sqrt(np.trapz(eta**2, x=gamma))
plt.loglog(kappa, relative_error)
plt.xlabel(r'$\kappa$')
plt.ylabel(r'$e$ for $\gamma\in(0,1/3)$')
plt.tight_layout()
plt.show()

## Weak potential

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
kappa_list = [1e1, 5e0, 1e-1]
eta = np.linspace(1e-3, 12e0, 100)
for kappa in kappa_list:
    gamma = fjc.nondimensional_end_to_end_length_per_link(eta/kappa, kappa)
    plt.plot(gamma, eta, label=r'$\kappa=$' + str(kappa))
    gamma_asymptotic = fjc.asymptotic.weak_potential.nondimensional_end_to_end_length_per_link(eta/kappa, kappa)
    plt.plot(gamma_asymptotic, eta, 'k--', label='asymptotic')
plt.legend()
plt.xlim([0, 0.4])
plt.ylim([0, 12])
plt.xlabel(r'$\gamma$')
plt.ylabel(r'$\eta$')
plt.show()